Instala la librería `contractions` para expandir contracciones en inglés.

In [ ]:
%pip install contractions

Importa las librerías necesarias para procesamiento y análisis de datos.

In [ ]:
import contractions
import os
import re
import matplotlib.pyplot as plt
import pandas as pd
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')

Define funciones para limpiar y parsear archivos XML de chats, y revisa errores de parseo en los archivos.

In [ ]:
def clean_invalid_xml_chars(filepath):
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()

    def clean_body_content(match):
        body_text = match.group(1)
        cleaned_text = re.sub(r'[<>&"\']', '', body_text)  # Elimina caracteres problemáticos
        return f"<BODY>{cleaned_text}</BODY>"

    content = re.sub(r'<BODY>(.*?)</BODY>', clean_body_content, content, flags=re.DOTALL | re.IGNORECASE)
    return content

def parse_chatlog(xml_file):
    try:
        content = clean_invalid_xml_chars(xml_file)
        root = ET.fromstring(content)

        predator_usernames = [sn.findtext('USERNAME') for sn in root.findall('PREDATOR/SCREENNAME')]
        victim_usernames = [sn.findtext('USERNAME') for sn in root.findall('VICTIM/SCREENNAME')]

        posts = []
        for post in root.findall('POST'):
            username = post.findtext("USERNAME")
            if not username or username.strip() == "":
                continue  # ⛔️ Saltar líneas sin username

            role = (
                'predator' if username in predator_usernames
                else 'victim' if username in victim_usernames
                else 'desconocido'
            )

            post_data = {
                'username': username.strip(),
                'datetime': post.findtext("DATETIME"),
                'body'    : post.findtext('BODY'),
                'role'    : role
            }

            posts.append(post_data)

        return pd.DataFrame(posts)

    except ET.ParseError as e:
        raise ET.ParseError(f"{xml_file} → {str(e)}")


# Revisar errores en los archivos xml

def revisar_errores_en_xmls(folder_path):
    errores = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".xml"):
            full_path = os.path.join(folder_path, filename)
            try:
                _ = parse_chatlog(full_path)
            except ET.ParseError as e:
                errores.append(str(e))

    if errores:
        print("\n❌ Archivos con errores de parseo:\n")
        for err in errores:
            print(f"  - {err}")
    else:
        print("✅ Todos los archivos XML fueron procesados sin errores.")

revisar_errores_en_xmls("GeneralData")

Carga todos los archivos XML, los procesa y combina en un solo DataFrame.

In [ ]:
# Ruta de tu carpeta con los XMLs
folder_path = "GeneralData"
all_dfs = []

for filename in os.listdir(folder_path):
    if filename.endswith(".xml"):
        full_path = os.path.join(folder_path, filename)
        df = parse_chatlog(full_path)
        if not df.empty:
            df['file'] = filename  # Para saber de qué archivo viene cada línea
            all_dfs.append(df)

# Combinar todos los DataFrames
combined_df = pd.concat(all_dfs, ignore_index=True)
print(combined_df.head())

Muestra las primeras filas del DataFrame combinado.

In [ ]:
combined_df.head()

Muestra las últimas filas del DataFrame combinado.

In [ ]:
combined_df.tail()

Define un diccionario de reemplazo para normalizar slangs y una función para normalizar texto.

In [ ]:
replacement_dict = {
    r"\bluv\b": "love",
    r"\bdoin\b": "doing",
    r"\bu\b": "you",
    r"\bru\b": "are you",
    r"\br u\b": "are you",
    r"\bya\b": "you",
    r"\byr\b": "your",
    r"\bur\b": "your",
    r"\br\b": "are",
    r"\bim\b": "i'm",
    r"\bidk\b": "i don't know",
    r"\bc\b": "see",
    r"\bc u\b": "see you",
    r"\bcya\b": "see you",
    r"\bbtw\b": "by the way",
    r"\bomg\b": "oh my god",
    r"\bthx\b": "thanks",
    r"\bpls\b": "please",
    r"\bplz\b": "please",
    r"\blol\b": "laughing out loud",
    r"\blmao\b": "laughing my ass off",
    r"\bwtf\b": "what the fuck",
    r"\bwth\b": "what the hell",
    r"\bomw\b": "on my way",
    r"\bl8r\b": "later",
    r"\bgr8\b": "great",
    r"\bmsg\b": "message",
    r"\btxt\b": "text",
    r"\bpic\b": "picture",
    r"\bttyl\b": "talk to you later",
    r"\bbrb\b": "be right back",
    r"\bafk\b": "away from keyboard",
    r"\bbf\b": "boyfriend",
    r"\bgf\b": "girlfriend",
    r"\bwanna\b": "want to",
    r"\bgonna\b": "going to",
    r"\bgotta\b": "got to",
    r"\bk\b": "okay",
    r"\bok\b": "okay",
    r"\bcuz\b": "because",
    r"\bcoz\b": "because",
    r"\btho\b": "though",
    r"\btho\b": "though",
    r"\btho\b": "though",
    r"\bdat\b": "that",
    r"\bda\b": "the",
    r"\bdis\b": "this",
    r"\bdem\b": "them",
    r"\bain't\b": "is not",
    r"\bgotcha\b": "got you",
    r"\bsorta\b": "sort of",
    r"\bkinda\b": "kind of",
    r"\bdunno\b": "don't know",
    r"\btryna\b": "trying to",
    r"\blemme\b": "let me",
    r"\bgimme\b": "give me",
    r"\bwhatcha\b": "what are you",
    r"\bill\b": "i will",
    r"\b2\b": "too",
    r"\b4\b": "for",
    r"\bme2\b": "me too",
    r"\bp\b": "",
    r"\bk\b": "ok",
    r"\bgetin\b": "getting",
    r"\bgtg\b": "got to go",
    r"\bu2\b": "you too",
    r"\b2day\b": "today",
    r"\bb4\b": "before",
    r"\b4u\b": "for you",
    r"\b2nite\b": "tonight",
    r"\b2moro\b": "tomorrow",
    r"\b4u\b": "for you",
    r"\b4ever\b": "forever",
    r"\bb4\b": "before",
    r"\b4get\b": "forget",
    r"\bgr8\b": "great",
    r"\b8\b": "ate",
    r"\b1\b": "one",
    r"\b1der\b": "wonder",
    r"\b1st\b": "first",
    r"\bcu\b": "see you",
    r"\bc u\b": "see you",
    r"\b4u\b": "for you",
    r"\bu\b": "you",
    r"\bur\b": "your",
    r"\br\b": "are",
    r"\bpls\b": "please",
    r"\bplz\b": "please",
    r"\bthx\b": "thanks",
    r"\bomg\b": "oh my god",
    r"\bcauz\b": "because",
    r"\bcuz\b": "because",
    r"\bhehe\b": "",
    r"\bcomdom\b": "condom",
    r"\bpreggerz\b": "pregnant",
    r"\bmin\b": "minutes",
    r"\bscard\b": "scared",
    r"\bhafta\b": "have to",
    r"\bprof\b": "profile",
    r"\bpromis\b": "promise",
    r"\bcallin\b": "calling",
    r"\bhav\b": "have",
    r"\bno\b": "no",
    r"\bhehe\b": "",
    r"\bsooo\b": "so",
    r"\byeah\b": "yes",
    r"\bwait\b": "wait",

}

def normalize_text(text):
    if not isinstance(text, str):  # Manejo de valores no-string (por si hay NaN)
        return ""

    # Paso 1: Convertir a minúsculas
    text = text.lower()

    # Paso 2: Expandir contracciones con la librería contractions
    text = contractions.fix(text)

    # Paso 3: Reemplazar slangs usando tu diccionario
    for pattern, replacement in replacement_dict.items():
        text = re.sub(pattern, replacement, text)

    # Paso 4: Limpieza adicional (opcional)
    text = re.sub(r'[^a-z0-9\s]', ' ', text) # Elimina caracteres no alfabéticos
    text = re.sub(r'\s+', ' ', text).strip()  # Elimina espacios extras

    return text

Aplica la normalización de texto a la columna 'body' y muestra los resultados.

In [ ]:
combined_df['normalized_body'] = combined_df['body'].apply(normalize_text)
# Mostrar resultados
combined_df['normalized_body']

In [ ]:
# Une todos los textos del cuerpo de los mensajes en una sola cadena
predator_text = ' '.join(combined_df[combined_df['role'] == 'predator']['normalized_body'].dropna().astype(str))

predator_text = predator_text.lower()

stop_words = set(stopwords.words('english'))

predator_text = re.sub(r'\d+', '', predator_text)
predator_text = re.sub(r'[^\w\s]', '', predator_text)

palabras = [word for word in predator_text.split() if word not in stop_words]

# Crea el objeto WordCloud
wordcloud = WordCloud(width=1600, height=800, background_color='white', collocations=False).generate(' '.join(palabras))

# Mostrar el WordCloud
plt.figure(figsize=(20, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("WordCloud de los groomers", fontsize=24)
plt.show()

In [ ]:

# Une todos los textos del cuerpo de los mensajes en una sola cadena
victim_text = ' '.join(combined_df[combined_df['role'] == 'victim']['normalized_body'].dropna().astype(str))

victim_text = victim_text.lower()

stop_words = set(stopwords.words('english'))

victim_text = re.sub(r'\d+', '', victim_text)
victim_text = re.sub(r'[^\w\s]', '', victim_text)

palabras = [word for word in victim_text.split() if word not in stop_words]

# Crea el objeto WordCloud
wordcloud = WordCloud(width=1600, height=800, background_color='white', collocations=False).generate(' '.join(palabras))

# Mostrar el WordCloud
plt.figure(figsize=(20, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("WordCloud de las victimas", fontsize=24)
plt.show()

Se ha visualizado varios "Slangs" que usan tanto los groomers como las victimas, por lo que se ha creado un diccionario para poder entender mejor las palabras usadas y pasarlas a palabras mas formales

 **Cantidad de mensajes de predadores y victimas en el archivo de ejemplo**

In [ ]:
conteo = combined_df['role'].value_counts()

conteo = conteo[conteo.index.isin(['victim', 'predator'])]

plt.figure(figsize=(8, 6))
plt.pie(conteo, labels=conteo.index, autopct='%1.1f%%', startangle=90, colors=['lightcoral', 'skyblue'])
plt.title('Distribución de Víctimas y Depredadores (ArmySgt1961.xml)')
plt.axis('equal')

plt.show()

Contador de la longitud de palabras en archivo *'ArmySgt1961.xml'*

In [ ]:
combined_df['word_length'] = combined_df['normalized_body'].str.split().str.len()

# Mostrar las primeras filas para verificar
print(combined_df[['role', 'normalized_body', 'word_length']].head())


In [ ]:
import seaborn as sns

# Filtrar datos
df_filtered = combined_df[combined_df['role'].isin(['victim', 'predator'])]

plt.figure(figsize=(10, 6))
sns.boxplot(
    x='role',
    y='word_length',
    hue='role',  # <- Añadimos hue para evitar el warning
    data=df_filtered,
    palette={'victim': 'lightcoral', 'predator': 'skyblue'},
    legend=False  # <- Ocultamos leyenda si no es necesaria
)
plt.title('Distribución de longitud de palabras')
plt.xlabel('Rol')
plt.ylabel('Número de palabras')
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
#plt.ylim(0, 20)   Mostrar solo mensajes con menos de 20 palabras

plt.show()

In [ ]:
pd.set_option('display.max_colwidth', None)  # Desactiva el recorte
print(combined_df[combined_df['role'] == 'predator'].nlargest(1, 'word_length')['normalized_body'])


In [ ]:

#combined_df['datetime'] = pd.to_datetime(combined_df['datetime'], errors='coerce')
dt_data = combined_df['datetime']

fecha_df = pd.DataFrame({'datetime_str': dt_data})

def preprocess_datetime(dt_str):

    if pd.isna(dt_str) or not isinstance(dt_str, str):
        return None

    #Sacar parentesis
    dt_str = dt_str.strip("()")

    #Sacar corchetes
    dt_str = dt_str.strip("[]")

    # Sacar espacios
    re.sub(r'\s+', ' ', dt_str).strip()

    # Estandarizar am/pm
    dt_str = dt_str.replace('a.m.', 'am').replace('p.m.', 'pm')
    dt_str = dt_str.replace('A.M.', 'AM').replace('P.M.', 'PM')

    if not dt_str:
        return None
    return dt_str

fecha_df['datetime_limpio'] = fecha_df['datetime_str'].apply(preprocess_datetime)
print(fecha_df[['datetime_str', 'datetime_limpio']].head(20))

In [ ]:
# Formato fecha y hora
formats_to_try = [
    "%m/%d/%Y %I:%M:%S %p",  # ej, 5/12/2006 1:53:15 pm
    "%m/%d/%y %I:%M:%S %p",  # ej, 10/01/08 9:43:46 pm
    "%I:%M:%S %p",           # ej, 7:14:04 pm
    "%I:%M %p"
]
fecha_df['datetime_parseado'] = pd.NaT

for i, row in fecha_df.iterrows():
    val_limpio = row['datetime_limpio']
    if pd.isna(val_limpio):
        continue

    parseado = False
    for fmt in formats_to_try:
        try:
            fecha_df.loc[i, 'datetime_parseado'] = pd.to_datetime(val_limpio, format=fmt)
            parseado = True
            break
        except ValueError:
            continue

print(fecha_df[['datetime_str', 'datetime_limpio', 'datetime_parseado']].head(20))

unparsed = fecha_df[fecha_df['datetime_parseado'].isna() & fecha_df['datetime_limpio'].notna()]
if not unparsed.empty:
    print(f"\n {len(unparsed)} Entradas no pudieron ser parseadas:")
    print(unparsed[['datetime_str', 'datetime_limpio']])
    print("\nEntradas no parseadas:")
    print(unparsed['datetime_limpio'].value_counts())
else:
    print("\nEntradas datetime no nulas ya parseadas o limpias.")

In [ ]:
fecha_df['datetime_parseado'] = pd.to_datetime(fecha_df['datetime_parseado'], errors='coerce')

# Extraer la hora del dia
fecha_df_valid_dates = fecha_df.dropna(subset=['datetime_parseado'])
fecha_df_valid_dates['hora_del_dia'] = fecha_df_valid_dates['datetime_parseado'].dt.hour

# Contar la actividad por hora
actividad_por_hora = fecha_df_valid_dates['hora_del_dia'].value_counts().sort_index()

# Graficar
plt.figure(figsize=(12, 6))
plt.bar(actividad_por_hora.index, actividad_por_hora.values, color='skyblue', width=0.8)

plt.title('Actividad por Hora del Día', fontsize=16)
plt.xlabel('Hora del Día (Formato 24h)', fontsize=12)
plt.ylabel('Número de Mensajes/Eventos', fontsize=12)
plt.xticks(ticks=range(0, 24), labels=[str(h) for h in range(0, 24)])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

print("\nTabla de actividad por hora del día:")
print(actividad_por_hora)

# Preparacion ML


In [ ]:
chats_agrupados = combined_df.groupby('file')['normalized_body'].apply(lambda msgs : ' '.join(msgs)).reset_index()
chats_agrupados.columns = ['file','full_chat']
chats_agrupados['label'] = 1


In [ ]:
grooming_chats_agrupados = chats_agrupados.drop(columns=['file'])

In [ ]:
from datasets import load_dataset

dataset = load_dataset("daily_dialog", trust_remote_code=True)

In [ ]:
dialog_data = dataset['train'].to_pandas()

In [ ]:
dialog_data['dialog'] = dialog_data['dialog'].apply(lambda x: ' '.join(eval(x)) if isinstance(x, str) and x.startswith('[') else ' '.join(x) if isinstance(x,list) else str(x))
dialog_data['normalized_dialog'] = dialog_data['dialog'].apply(normalize_text)

In [ ]:
dialog_data = dialog_data.drop(columns=['act'])
dialog_data = dialog_data.drop(columns=['emotion'])
dialog_data['label'] = 0

In [ ]:
dialog_chats_agrupados = dialog_data[['normalized_dialog']].copy()
dialog_chats_agrupados.rename(columns={'normalized_dialog': 'full_chat'}, inplace=True)
dialog_chats_agrupados['label'] = 0


In [ ]:
datos_para_ml = pd.concat([grooming_chats_agrupados, dialog_chats_agrupados], ignore_index=True)


In [26]:
output_filename = 'datos_finales_para_ml.csv'
try:
    datos_para_ml.to_csv(output_filename, index=False)
    print(f"\n DF final para ML guardado en: {output_filename}")
except Exception as e:
    print(f"Error al guardar el archivo: {e}")


 DF final para ML guardado en: datos_finales_para_ml.csv
